# Portable Power BI Dashboard Templates with Fabric Notebooks

**Proof of Concept:** Transfer of a Power BI dashboard template to another semantic model  
**Author:** Marcus Wegener • GitHub: [pbi-dashboard-template-notebook](https://github.com/MarcusWegener/pbi-dashboard-template-notebook)

> **What this notebook demonstrates**  
> A parameterized approach to *re-binding* field references in a Power BI report definition so that a dashboard template can be redeployed to a **different** semantic model with minimal manual effort.  
> This enables a reusable **dashboard catalog** or even a **dashboard factory** pattern.

---

## Why this matters
Power BI report definitions often contain hard-coded field bindings (tables, columns, measures). Reusing reports across models is tedious and error-prone.  
This notebook shows how to **systematically replace** those bindings using Fabric Notebooks and `semPy`.

---

## At a glance
- ✅ Parameterized template → **portable** dashboards
- ✅ Uses `semPy` to update report definition programmatically
- ✅ Keeps calculations **loosely coupled** using *Visual Calculations* where appropriate
- ⚠️ **No sensitive data** required — uses a public reference model and demo data

---

## Prerequisites (Fabric)
- Fabric capacity (trial or paid) and a workspace in Fabric
- Contributor (or higher) on the workspace
- Build permission for any semantic model you connect to
- `semPy` enabled (Python in notebooks is currently **Preview**; PySpark is **GA**)
- Your semantic model name must **not** end with a space

> This notebook builds on the published reference project and template:  
> **GitHub:** https://github.com/MarcusWegener/pbi-dashboard-template-notebook


In [ ]:
import io, os, json, base64, zipfile, tempfile, pathlib, typing, requests
import sempy.fabric as fabric
from sempy.fabric import FabricRestClient

# ---------- CONFIG ----------
workspace_id     = "05407033-60c6-4fdc-85db-1d126ba9be51"

# Source (GitHub)
github_repo      = "https://github.com/MarcusWegener/pbi-dashboard-template-notebook"
github_branch    = "main"
pbip_subfolder   = ""  # e.g., "src/powerbi"; leave empty if PBIP is at repo root

# Target Report
report_name_override = "Monitoring Dashboard Country"  # Optional: enforce a display name in the Service; else derived from folder

# Dataset binding (REQUIRED: use the Semantic Model item ID in the target workspace)
DATASET_ID       = "79d647a8-9f43-4317-899a-660c0eb8999d"

# ---------- REPLACEMENT CONFIG (for visual.json) ----------
# Example mappings:
act_measure = "Sales.Net Sales Amount by Order Date"
bud_measure = "Sales.Net Sales Amount by Order Date PY"
dimension   = "Customer.Country Name"

REPLACEMENTS = {
    # explicit tokens
    "FactACTTable.ACTMeasure": act_measure,
    "FactBUDTable.BUDMeasure": bud_measure,
    "FactACTTable":         act_measure.split(".")[0],     # left side of act_measure
    "ACTMeasure":           act_measure.split(".")[1],     # right side of act_measure
    "FactBUDTable":         bud_measure.split(".")[0],     # left side of bud_measure
    "BUDMeasure":           bud_measure.split(".")[1],     # right side of bud_measure
    "DimTable.DimColumn":   dimension,
    "DimTable":             dimension.split(".")[0],       # left side of dimension
    "DimColumn":            dimension.split(".")[1],       # right side of dimension
}

# ---------- HELPERS ----------
def dl_and_extract_zip(zip_url: str) -> pathlib.Path:
    """Download repo ZIP and extract to a temp folder. Return the top-level folder."""
    tmpdir = tempfile.mkdtemp(prefix="pbip_report_")
    resp = requests.get(zip_url); resp.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(resp.content)) as zf:
        zf.extractall(tmpdir)
    roots = [pathlib.Path(tmpdir)/d for d in os.listdir(tmpdir)]
    return next((p for p in roots if p.is_dir()), pathlib.Path(tmpdir))

def read_pbir(report_dir: pathlib.Path) -> dict:
    """Load definition.pbir as JSON."""
    p = report_dir / "definition.pbir"
    if not p.exists():
        raise FileNotFoundError(f"definition.pbir not found under {report_dir}")
    return json.loads(p.read_text(encoding="utf-8"))

def b64_bytes(data: bytes) -> str:
    return base64.b64encode(data).decode("utf-8")

def rewrite_visual_json(report_dir: pathlib.Path, replacements: dict) -> None:
    """
    Walk through ALL visual.json files under the report's 'definition' folder
    and replace configured identifiers with new ones (simple string replace).
    """
    def_root = report_dir / "definition"
    if not def_root.exists():
        raise FileNotFoundError(f"'definition' folder is missing in report: {report_dir}")
    count = 0
    for vfile in def_root.rglob("visual.json"):
        text = vfile.read_text(encoding="utf-8")
        for old, new in replacements.items():
            text = text.replace(old, new)
        vfile.write_text(text, encoding="utf-8")
        count += 1
    print(f"Rewrote identifiers in {count} visual.json file(s).")

def make_report_parts(report_dir: pathlib.Path, pbir_obj: dict) -> list:
    """
    Build PBIR parts:
      - 'definition.pbir' (at report root)
      - ALL files under 'definition/' (incl. 'definition/report.json' and assets)
    Paths must mirror the PBIP layout.
    """
    parts = []
    # 1) definition.pbir
    raw_pbir = json.dumps(pbir_obj, ensure_ascii=False, indent=2).encode("utf-8")
    parts.append({"path": "definition.pbir", "payload": b64_bytes(raw_pbir), "payloadType": "InlineBase64"})

    # 2) entire 'definition/' folder
    def_root = report_dir / "definition"
    if not def_root.exists():
        raise FileNotFoundError(f"'definition' folder is missing in report: {report_dir}")
    for p in def_root.rglob("*"):
        if p.is_file():
            rel = str(p.relative_to(report_dir)).replace("\\", "/")  # 'definition/...'
            parts.append({"path": rel, "payload": b64_bytes(p.read_bytes()), "payloadType": "InlineBase64"})
    return parts

def get_report_display_name(report_dir: pathlib.Path) -> str:
    return (report_name_override or report_dir.stem.replace(".Report", "")).strip()

def ensure_by_connection_always(pbir_obj: dict, target_dataset_id: str) -> dict:
    """
    Always overwrite datasetReference with a byConnection pointing to the given dataset ID.
    """
    if not target_dataset_id or not target_dataset_id.strip():
        raise RuntimeError("DATASET_ID is required and must not be empty.")
    pbir_obj["datasetReference"] = {
        "byConnection": {
            "pbiModelVirtualServerName": "sobe_wowvirtualserver",
            "pbiModelDatabaseName": target_dataset_id.strip(),
            "name": "EntityDataSource",
            "connectionType": "pbiServiceXmlaStyleLive",
            "connectionString": None,
            "pbiServiceModelId": None
        }
    }
    return pbir_obj

# ---------- MAIN ----------
zip_url = f"{github_repo}/zipball/{github_branch}"
root = dl_and_extract_zip(zip_url)
project_root = root / pbip_subfolder if pbip_subfolder else root

# Find the first PBIP report folder
report_dir = next(project_root.glob("**/*.Report"), None)
if report_dir is None:
    raise RuntimeError(f"No *.Report folder found under {project_root}")

display_name = get_report_display_name(report_dir)

# Load PBIR and force byConnection binding to the specified dataset
pbir = read_pbir(report_dir)
pbir = ensure_by_connection_always(pbir, DATASET_ID)

# Apply identifier replacements in all visual.json files BEFORE packaging parts
rewrite_visual_json(report_dir, REPLACEMENTS)

# Build PBIR parts (definition.pbir + full 'definition/' folder)
parts = make_report_parts(report_dir, pbir)

# Create report via FabricRestClient (auth handled by the notebook environment)
client = FabricRestClient()

print(f"Creating report '{display_name}' ...")
url = f"v1/workspaces/{workspace_id}/items"
payload = {
    "type": "Report",
    "displayName": display_name,
    "definition": {"format": "PBIR", "parts": parts}
}
resp = client.post(url, json=payload, lro_wait=True)

# Parse response safely
try:
    body = resp.json()
except Exception:
    body = {}

item_id = body.get("id") or (body.get("item") or {}).get("id") or (body.get("operation") or {}).get("targetId")
print("Created. HTTP:", resp.status_code, "ItemId:", item_id or "unknown")
print("=== DONE ===")


## ⚠️ Disclaimer

This notebook is provided **as-is** for demonstration purposes only. 
Use at your own risk; **no liability** is assumed for any damages arising from its use.


## Design Notes & Limitations

- **Visual Calculations** can store logic *inside visuals*, reducing coupling to the semantic model and improving portability.  
- **Report-specific measures** are possible. For parametrization, define a dedicated **home table** for report measures.  
- The `visual.json` also references a **home table** for measures. Therefore, **template placeholders must be unique and consistent**, so measures bound to different home tables remain distinguishable and correctly mapped.

> These constraints inform how the **reference model** and the **template** should be designed to be truly portable.


## How to Reuse with a Different Semantic Model

1. **Clone / open** this notebook in a Fabric workspace backed by capacity.  
2. **Point parameters** (model, dataset, table/measure placeholders) to the **target** semantic model.  
3. **Run** the notebook
4. **Validate** result (open the report)
5. **Iterate**: if a binding fails, adjust the mapping table or template placeholders.

> Tip: Keep a **mapping registry** (CSV/JSON) for each target model so deployments are repeatable.


## Credits & Links

- Inspired by **FUAM deployment** (Kevin Thomas, Gellért Gintli)  
- Inspired by **Patrick LeBlanc’s** video: *Creative way to use semPy to update a Power BI report definition*  
- Project & reference model: **https://github.com/MarcusWegener/pbi-dashboard-template-notebook**

- Patrick LeBlanc: *Creative way to use semPy to update a Power BI report definition*  
  https://youtu.be/HT_J1QiBLwA?si=MpxYYYWvGIBtUhTM
- Fabric Unified Admin Monitoring (FUAM)  
  https://github.com/microsoft/fabric-toolbox/tree/main/monitoring/fabric-unified-admin-monitoring
